In [1]:
from datetime import datetime
import pandas as pd 
import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.express as px
from scipy.stats import lognorm
from statsmodels.distributions.empirical_distribution import ECDF


In [2]:
def rolling_vpin(df_rolling_volumebar, n = 200) : 
    
    df_rolling_vpin = df_rolling_volumebar.copy()

    # Rolling Sum of N Volume Bars
    df_rolling_vpin['V_rollingsum'] = df_rolling_vpin['V'].rolling(n).sum()

    # Original VPIN
    df_rolling_vpin['buy_sell_diff'] = df_rolling_vpin['buy_volume'] - df_rolling_vpin['sell_volume']
    df_rolling_vpin['buy_sell_diff_abs'] = abs(df_rolling_vpin['buy_sell_diff'])
    df_rolling_vpin['buy_sell_diff_abs_rollingsum'] = df_rolling_vpin['buy_sell_diff_abs'].rolling(n).sum()
    df_rolling_vpin['vpin'] = (df_rolling_vpin['buy_sell_diff_abs_rollingsum']/df_rolling_vpin['V_rollingsum'])

    # VPIN Sell
    df_rolling_vpin['diff_only_sell'] = np.where(df_rolling_vpin['buy_sell_diff'] > 0, 0, df_rolling_vpin['buy_sell_diff'])
    df_rolling_vpin['diff_only_sell_abs'] = abs(df_rolling_vpin['diff_only_sell'])
    df_rolling_vpin['diff_only_sell_abs_rollingsum'] = df_rolling_vpin['diff_only_sell_abs'].rolling(n).sum()
    df_rolling_vpin['vpin_sell'] = (df_rolling_vpin['diff_only_sell_abs_rollingsum']/df_rolling_vpin['V_rollingsum'])
    
    # VPIN Buy
    df_rolling_vpin['diff_only_buy'] = np.where(df_rolling_vpin['buy_sell_diff'] < 0, 0, df_rolling_vpin['buy_sell_diff'])
    df_rolling_vpin['diff_only_buy_abs'] = abs(df_rolling_vpin['diff_only_buy'])
    df_rolling_vpin['diff_only_buy_abs_rollingsum'] = df_rolling_vpin['diff_only_buy_abs'].rolling(n).sum()
    df_rolling_vpin['vpin_buy'] = (df_rolling_vpin['diff_only_buy_abs_rollingsum']/df_rolling_vpin['V_rollingsum'])

    # df_rolling_vpin['ratio'] = df_rolling_vpin['buy_sell_diff_abs']/ (df_rolling_vpin['V']/n)
    # df_rolling_vpin['vpin'] = df_rolling_vpin['ratio'].rolling(n).sum()

    df_rolling_vpin = df_rolling_vpin[['open_time', 'vpin', 'vpin_sell']]

    return df_rolling_vpin

In [3]:
df_rolling_volumebar_btc = pd.read_pickle("df_rolling_volumebar.pkl")
df_vpin = rolling_vpin(df_rolling_volumebar_btc, 200)

In [9]:
dist_window = 600
vpin_col = 'vpin_sell'
df_vpin['vpin_low'] = df_vpin[vpin_col].rolling(window= dist_window).quantile(0.1)
df_vpin['vpin_med'] = df_vpin[vpin_col].rolling(window= dist_window).quantile(0.2)
df_vpin['vpin_high'] = df_vpin[vpin_col].rolling(window= dist_window).quantile(0.3)


In [10]:
df_vpin.dropna().head(15)

,open_time,vpin,vpin_sell,vpin_low,vpin_med,vpin_high
798,2018-12-05 01:13:00,0.063593,0.024124,0.012622,0.013849,0.014848
799,2018-12-05 04:11:00,0.064216,0.024763,0.012622,0.013850,0.014858
800,2018-12-05 07:55:00,0.063973,0.024536,0.012622,0.013872,0.014864
801,2018-12-05 11:21:00,0.063697,0.024615,0.012622,0.013907,0.014867
802,2018-12-05 14:35:00,0.063651,0.024605,0.012622,0.013927,0.014871
803,2018-12-05 16:54:00,0.064126,0.025096,0.012622,0.013935,0.014874
804,2018-12-05 17:31:00,0.065004,0.026032,0.012622,0.013941,0.014878
805,2018-12-05 20:59:00,0.064627,0.025671,0.012622,0.013948,0.014880
806,2018-12-06 01:06:00,0.064447,0.025711,0.012622,0.013954,0.014884
807,2018-12-06 04:51:00,0.064118,0.025794,0.012622,0.013960,0.014887


In [15]:
df_vpin = df_vpin.dropna().reset_index(drop = True)
df_vpin.to_csv('vpin.csv')